In [1]:
# genetic algorithm search of the one max optimization problem
import numpy as np
from numpy.random import randint
from numpy.random import rand
import matplotlib.pyplot as plt
import math
import cv2
import os

## Objective function

In [2]:
def color_match(x, target):
	dist = 0
	for c1, c2 in zip(x, target):
		dist += (c1-c2)**2
	return math.sqrt(dist)

## Elite+Roulette selection

In [3]:
import pandas as pd

def scores2prob(scores):
    inv_scores = [ 1/(score+ 1e-10) for score in scores]
    agg = sum(inv_scores)
    prob = [ score/agg for score in inv_scores]
    
    return prob

def selection(pop, scores):
    
    nsamples = len(pop)
    df = pd.DataFrame({
          'pop': pop,
          'scores': scores
          })
    
    df = df.sort_values(by=['scores']).reset_index(drop=True) # sorted in ascending order
    new_parents = df['pop'] #.tolist()[:top_k]
    new_scores = df['scores'] #.tolist()[:top_k]
    prob = scores2prob(new_scores)
    
    sample_idx = [np.random.choice(range(len(new_parents)), 2, replace=False, p=prob).tolist() for _ in range(nsamples)]

    ldict_selection = []
    for p1_idx, p2_idx in sample_idx:
        agg = prob[p1_idx]+prob[p2_idx]
        d_sel = {
            'parents': [new_parents[p1_idx], new_parents[p2_idx]],
            'scores': [prob[p1_idx]/agg, prob[p2_idx]/agg]
            }
        ldict_selection.append(d_sel)
	
    return ldict_selection


## Arithmetic crossover

In [4]:
def crossover(ldict_selection):
	l_child = []
	for d_sel in ldict_selection:
		p1, p2 = d_sel['parents']
		s1, s2 = d_sel['scores']
		child = (s1 * np.array(p1) + s2 * np.array(p2)).astype(np.uint8).tolist() #arithmetic crossover

		l_child.append(child)
	
	return l_child

## Mutation operator

In [5]:
from copy import deepcopy

def mutation(l_child, r_mut):
	l_mutation = deepcopy(l_child)
	for sidx, sample in enumerate(l_mutation):
		for cidx, clr in enumerate(sample):
			# check for a mutation
			if rand() < r_mut: # P(0<.= r <=1.0) < r_mut
				mask = np.uint8(1 << randint(0, 8))
				sample[cidx] = clr ^ mask # XOR prperty 0 ^ 0 = 0, 0 ^ 1 = 1, 1 ^ 0 = 1, 1^ 1=0 
		l_mutation[sidx] = sample
	return l_mutation


## Display the interation results

In [6]:
def display_pallete(sample_pop, target_color, 
                    children, mutations, best, generation,
                    n_row, n_col):
    pop_palette = np.array(sample_pop)[np.newaxis, :, :].reshape(n_row, n_col, 3)
    chd_palette = np.array(children)[np.newaxis, :, :].reshape(n_row, n_col, 3)
    mut_palette = np.array(mutations)[np.newaxis, :, :].reshape(n_row, n_col, 3)
    
    target_pal = np.array([target_color])[np.newaxis, :, :]
    best_pal = np.array([best])[np.newaxis, :, :]

    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(12, 8))
    fig.patch.set_facecolor('white')

    ax[0].imshow(target_pal)
    ax[0].axis('off')
    ax[0].title.set_text('Target')

    ax[1].imshow(pop_palette)
    ax[1].axis('off')
    ax[1].title.set_text(f'Generation:{str(generation).zfill(3)} parents')

    ax[2].imshow(chd_palette)
    ax[2].axis('off')
    ax[2].title.set_text(f'Generation:{str(generation).zfill(3)} children')

    ax[3].imshow(mut_palette)
    ax[3].axis('off')
    ax[3].title.set_text(f'Generation:{str(generation).zfill(3)} mutations')

    ax[4].imshow(best_pal)
    ax[4].axis('off')
    ax[4].title.set_text(f'Generation:{str(generation).zfill(3)} best result')

    plt.tight_layout()
    plt.savefig(f'./output/results_{str(generation).zfill(3)}.png')
    plt.close()


def createVideo():
	
    image_folder = 'output'
    video_name = './output/video.avi'
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
	
    images.sort()
	
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 1, (width,height))
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))
	
    cv2.destroyAllWindows()
    
    video.release()


## Genetic algorithm

In [7]:
def genetic_algorithm(pop, target,
					  objective, n_iter, r_mut, d_disp_grid_sz):
	
	n_pop = len(pop)
	# keep track of best solution
	best, best_eval = pop[0], objective(pop[0], target)
	
	# iterate over the generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		scores = [objective(c, target) for c in pop]
		
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		
		# select parents
		selected = selection(pop, scores)

		# create the next generation
		children = crossover(selected)
		
		# create mutations for gene diversity
		mutations = mutation(children, r_mut)

		# Display results from the generation
		display_pallete(pop, target, children, mutations, best, gen,
				  d_disp_grid_sz['N_ROW'], d_disp_grid_sz['N_COL'])

		# replace population
		pop = mutations
	
	createVideo()
	
	return [best, best_eval]


## Create initial population (color pallete in this case)

In [8]:
N_ROW, N_COL = 5, 10 # Plot rows, cols in color palette
N_POPULATION = N_ROW * N_COL
sample_pop = [tuple(randint(128, 255, 3)) for _ in range(N_POPULATION)] # Pick a triple of RGB values. Sample from higher values to get lighter shades
target_color = (0, 0, 0) #Delibrately picking black. tuple(randint(0, 255, 3))

n_iter = 150 # No.of generations
r_mut = 0.05 # Percent chance of mutation in the feature/gene
d_disp_grid_sz = {
    'N_ROW' : N_ROW,
    'N_COL' : N_COL
}

if not os.path.exists('./output/'):
    os.makedirs('./output/')

genetic_algorithm(sample_pop, target_color,
					  color_match, n_iter, r_mut, d_disp_grid_sz)

>0, new best f((183, 134, 135)) = 263.951
>2, new best f([195, 14, 172]) = 260.394
>3, new best f([183, 173, 40]) = 254.986
>6, new best f([50, 169, 169]) = 244.176
>7, new best f([57, 160, 164]) = 236.104
>9, new best f([169, 146, 38]) = 226.541
>11, new best f([177, 136, 24]) = 224.502
>12, new best f([183, 82, 30]) = 202.763
>14, new best f([51, 125, 134]) = 190.216
>23, new best f([146, 91, 52]) = 179.725
>24, new best f([145, 91, 27]) = 173.306
>25, new best f([18, 106, 125]) = 164.879
>26, new best f([23, 108, 113]) = 157.994
>38, new best f([54, 99, 110]) = 157.534
>39, new best f([120, 32, 92]) = 154.557
>40, new best f([110, 94, 45]) = 151.529
>44, new best f([54, 87, 103]) = 145.238
>49, new best f([107, 80, 43]) = 140.350
>52, new best f([36, 85, 103]) = 138.311
>54, new best f([104, 84, 34]) = 137.942
>56, new best f([43, 82, 101]) = 137.018
>57, new best f([94, 21, 96]) = 135.989
>57, new best f([71, 81, 83]) = 135.982
>60, new best f([98, 82, 32]) = 131.727
>65, new best 

[[26, 8, 17], 32.07802986469088]